In [1]:
import pandas as pd
import numpy as np
import gcsfs
import sys
import xarray as xr

In [2]:
df = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
# define a simple search on keywords
def search_df(df, verbose= False, **search):
    "search by keywords - if list, then match exactly, otherwise match as substring"
    keys = ['activity_id','institution_id','source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label']
    d = df
    for skey in search.keys():
        
        if isinstance(search[skey], str):  # match a string as a substring
            d = d[d[skey].str.contains(search[skey])]
        else:
            dk = []
            for key in search[skey]:       # match a list of strings exactly
                dk += [d[d[skey]==key]]
            d = pd.concat(dk)
            keys.remove(skey)
    if verbose:
        for key in keys:
            print(key,' = ',list(d[key].unique()))      
    return d

In [15]:
#dfp = search_df(df, experiment_id=['historical','ssp585'], table_id=['Omon','Amon'], variable_id=["uas","vas","tos"], grid_label=['gr'])
dfp = search_df(df, experiment_id=['historical','ssp585'], member_id=['r1i1p1f1'],table_id=["Amon", "Amon", "Omon", "Amon", "SImon", "SImon", "SImon",
                          "SImon"], variable_id=["uas", "vas", "chl", "clt", "sithick", "siconc", "sisnthick", "sisnconc"], grid_label=['gn'])
#dfp = search_df(df, experiment_id=['historical','ssp585'], member_id=['r1i1p1f1'],table_id=['Amon','SImon','Omon','Amon'], variable_id=["uas","vas","chl","clt"], grid_label=['gn'])


In [16]:
dm = dfp[['experiment_id','source_id','variable_id','member_id',]].groupby(['experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm, values='member_id', index=['experiment_id','source_id'],
                                 columns=['variable_id'], aggfunc=np.sum, fill_value=0)
table

variable_id                    chl  clt  siconc  sisnconc  sisnthick  sithick  \
experiment_id source_id                                                         
historical    ACCESS-CM2         0    1       1         1          1        1   
              ACCESS-ESM1-5      1    1       1         1          1        1   
              AWI-CM-1-1-MR      0    1       0         1          1        1   
              AWI-ESM-1-1-LR     0    1       0         0          0        0   
              BCC-CSM2-MR        0    1       1         0          1        0   
              BCC-ESM1           0    1       1         0          1        0   
              CAMS-CSM1-0        0    1       1         0          1        0   
              CAS-ESM2-0         0    0       0         0          0        0   
              CESM2              1    1       1         0          1        1   
              CESM2-FV2          1    1       1         0          1        1   
              CESM2-WACCM        1    1       1         0          1        1   
              CESM2-WACCM-FV2    1    1       1         0          1        1   
              CMCC-CM2-SR5       0    1       0         0          0        0   
              CanESM5            1    1       1         1          1        0   
              EC-Earth3          0    0       1         0          1        1   
              EC-Earth3-Veg      0    0       1         0          1        1   
              FGOALS-f3-L        0    0       1         0          0        0   
              FGOALS-g3          0    1       1         0          0        0   
              FIO-ESM-2-0        0    0       1         1          1        0   
              GFDL-CM4           0    0       1         1          1        1   
              GFDL-ESM4          0    0       1         1          1        1   
              GISS-E2-1-G        0    1       0         0          0        1   
              GISS-E2-1-G-CC     1    1       0         0          0        1   
              GISS-E2-1-H        0    1       0         0          0        1   
              IPSL-CM6A-LR       1    0       1         0          1        1   
              MCM-UA-1-0         0    0       0         0          0        1   
              MIROC6             0    1       1         0          1        1   
              MPI-ESM-1-2-HAM    1    1       1         1          1        1   
              MPI-ESM1-2-HR      1    1       1         1          1        1   
              MPI-ESM1-2-LR      1    1       1         1          1        1   
              MRI-ESM2-0         0    1       1         1          1        1   
              NESM3              0    1       1         0          1        1   
              NorCPM1            0    1       1         1          1        1   
              NorESM2-LM         0    1       1         0          1        1   
              NorESM2-MM         0    1       0         0          0        0   
              SAM0-UNICON        0    1       1         1          1        1   
              TaiESM1            0    1       1         1          1        0   
ssp585        ACCESS-CM2         0    1       1         1          1        1   
              ACCESS-ESM1-5      1    1       1         1          1        1   
              AWI-CM-1-1-MR      0    1       0         1          1        1   
              BCC-CSM2-MR        0    1       1         0          1        0   
              CAMS-CSM1-0        0    1       1         0          1        0   
              CESM2-WACCM        1    1       1         0          1        1   
              CMCC-CM2-SR5       0    0       0         0          0        0   
              CanESM5            1    1       1         1          1        0   
              EC-Earth3          0    0       1         0          1        1   
              EC-Earth3-Veg      0    0       1         0          1        1   
              FGOALS-g3        

In [17]:
table[(table.uas>0)&(table.vas>0)&(table.chl>0)&(table.clt>0)&(table.sisnconc>0)&(table.sisnthick>0)&(table.siconc>0)&(table.sithick>0)]

variable_id                    chl  clt  siconc  sisnconc  sisnthick  sithick  \
experiment_id source_id                                                         
historical    ACCESS-ESM1-5      1    1       1         1          1        1   
              MPI-ESM-1-2-HAM    1    1       1         1          1        1   
              MPI-ESM1-2-HR      1    1       1         1          1        1   
              MPI-ESM1-2-LR      1    1       1         1          1        1   
ssp585        ACCESS-ESM1-5      1    1       1         1          1        1   
              MPI-ESM1-2-HR      1    1       1         1          1        1   
              MPI-ESM1-2-LR      1    1       1         1          1        1   

variable_id                    uas  vas  
experiment_id source_id                  
historical    ACCESS-ESM1-5      1    1  
              MPI-ESM-1-2-HAM    1    1  
              MPI-ESM1-2-HR      1    1  
              MPI-ESM1-2-LR      1    1  
ssp585        ACCESS-ESM1-5      1    1  
              MPI-ESM1-2-HR      1    1  
              MPI-ESM1-2-LR      1    1

In [ ]:
def get_and_organize_cmip6_data(conf):
    # Dictionary to hold the queried variables
    first = True
    for experiment_id in conf.experiment_ids:
        for grid_label in conf.grid_labels:
            for source_id in conf.source_ids:
                for member_id in conf.member_ids:
                    for variable_id, table_id in zip(conf.variable_ids, conf.table_ids):
                        
                        # Create unique key to hold dataset in dictionary
                        key="{}_{}_{}_{}_{}".format(variable_id,experiment_id,grid_label,source_id,member_id)
                        # Historical query string
                        query_string = "source_id=='{}'and table_id=='{}' and grid_label=='{}' and experiment_id=='historical' and variable_id=='{}'".format(source_id, 
                        table_id, 
                        grid_label,
                        variable_id)
                        
                        print(
                            "Running historical query on data: \n ==> {}\n".format(query_string)
                        )
                        ds_hist = perform_cmip6_query(conf,query_string)
                       
                        # Future projection depending on choice in experiment_id
                        query_string = "source_id=='{}'and table_id=='{}' and member_id=='{}' and grid_label=='{}' and experiment_id=='{}' and variable_id=='{}'".format(
                                source_id,
                                table_id,
                                member_id,
                                grid_label,
                                experiment_id,
                                variable_id,
                            )
                            print(
                                "Running projections query on data: \n ==> {}\n".format(
                                    query_string
                                )
                            )
                        ds_proj = perform_cmip6_query(conf,query_string)

                        if first:
                            df_area = conf.df.query(
                                    "variable_id == 'areacella' and source_id =='{}'".format(
                                        source_id
                                    )
                                )
                            ds_area = xr.open_zarr(
                                    conf.fs.get_mapper(df_area.zstore.values[0]), consolidated=True
                                )
                            first = False

                        # Concatentate the historical and projections datasets
                    #    ds_hist=ds_hist.sel(time=slice(ds_hist["time"][0],"2000-12-15"))
                        #  print("Time in projection {} - {}".format(ds_proj["time"][0],ds_proj["time"][-1]))
                        ds = xr.concat([ds_hist, ds_proj], dim="time")

                        # Remove the duplicate overlapping times (e.g. 2001-2014)
                        #  _, index = np.unique(ds["time"], return_index=True)
                        #  ds = ds.isel(time=index)

                        # Extract the time period of interest
                        ds=ds.sel(time=slice(conf.start_date,conf.end_date))
                        print("{} => Dates extracted range from {} to {}\n".format(source_id,ds["time"].values[0], ds["time"].values[-1]))

                        # Save the dataset for variable_id in the dictionary
                        conf.dset_dict[key] = ds

def perform_cmip6_query(conf,query_string):
    df_sub = conf.df.query(query_string)
    if (df_sub.zstore.values.size==0):
        return df_sub
    
    mapper = conf.fs.get_mapper(df_sub.zstore.values[-1])
    ds = xr.open_zarr(mapper, consolidated=True)

    time_object = ds["time"].values[0]
    
    # Convert if necesssary
    if time_object.year == 1:
        
        times = ds["time"].values
        times_plus_2000 = []
        for t in times:
            times_plus_2000.append(
                cftime.DatetimeNoLeap(t.year + 2000, t.month, t.day, t.hour)
            )
        ds["time"].values = times_plus_2000
        ds = xr.decode_cf(ds)                    
    return ds

In [ ]:
class Config_pices():
    df = pd.read_csv(
    "https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv"
)
    fs = gcsfs.GCSFileSystem(token="anon", access="read_only")
   
    grid_labels = ["gr"]  # Can be gr=grid rotated, or gn=grid native
    member_ids = ["r1i1p1f1"]  #
    experiment_ids = ["ssp585"]  #'abrupt-4xCO2',
    source_ids = ["KACE-1-0-G"]
    variable_ids = ["tos","uas","vas"]
    table_ids = ["Omon","Amon","Amon"]  # Amon=atmospheric variables, Omon=Ocean variables, SImon=sea-ice variables
    dset_dict = {}
    start_date="1900-01-01"
    end_date="2100-08-01"

In [ ]:
config_pices_obj=Config_pices()
get_and_organize_cmip6_data(config_pices_obj)